# Test Belenguer format converter

In [7]:
%load_ext autoreload
%autoreload 2

import os
import sys
import importlib
import cProfile
import pandas as pd
sys.path.insert(0, os.path.abspath('../../'))
import logging
logging.basicConfig(level=logging.DEBUG)
logging.debug('This will get logged')

import tables as tb

DEBUG:root:This will get logged


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
from converter.input_converter import Converter
from converter.import_Belenguer_format import convert_file
import converter.py_data_write as py_data_write
import converter.shortest_paths as shortest_paths
from converter import load_instance
from converter import Network

In [9]:
file = '../../data/Lpr_IF/Lpr_IF-c-05.txt'
file_path = file

In [10]:
%%time
conv = Converter(file_path, out_to_in=True, overwrite=True)

CPU times: user 11 µs, sys: 1 µs, total: 12 µs
Wall time: 15 µs


In [11]:
%%time
instance_info = conv.extract_inputs()

'Converting ../../data/Lpr_IF/Lpr_IF-c-05.txt...'
CPU times: user 582 ms, sys: 71.2 ms, total: 653 ms
Wall time: 586 ms


In [12]:
success_array = np.array([np.array(x) for x in instance_info['sucArcL']])
instance_df = pd.DataFrame({'arc_u': instance_info['beginL'], 'arc_v': instance_info['endL'], 'arc_cost': instance_info['travelCostL']})
instance_df['arc_index'] = instance_df.index.values
instance_df['arc_inverse_index'] = instance_info['invArcL']
instance_df['arc_inverse_index'] = instance_df['arc_inverse_index'].fillna(-1).astype(int)
instance_df.loc[instance_df['arc_inverse_index'] == -1, 'arc_oneway'] = True
instance_df.loc[instance_df['arc_inverse_index'] != -1, 'arc_oneway'] = False
instance_df['arc_successor_index_list'] = np.nan
instance_df['arc_successor_index_list'] = instance_df['arc_successor_index_list'].astype(object)
instance_df.at[:, 'arc_successor_index_list'] = success_array

In [13]:
instance_df2 = convert_file(file)

'Converting ../../data/Lpr_IF/Lpr_IF-c-05.txt...'


In [14]:
pd._testing.array_equivalent(instance_df, instance_df2)

True

In [15]:
instance_df2

,arc_u,arc_v,arc_cost,arc_index,arc_inverse_index,arc_oneway,arc_successor_index_list
0,1,1,0,0,0,False,"[0, 264, 265, 266, 869]"
1,184,184,0,1,1,False,"[1, 248, 249, 852, 853]"
2,368,368,0,2,2,False,"[2, 454, 455, 1162]"
3,2,3,12,3,-1,True,"[41, 42, 463]"
4,7,8,19,4,-1,True,"[45, 473, 475]"
...,...,...,...,...,...,...,...
1226,365,364,16,1226,1225,False,"[450, 1224, 1225]"
1227,365,366,23,1227,1228,False,"[1228, 1229]"
1228,366,365,23,1228,1227,False,"[451, 452, 1226, 1227]"
1229,366,367,20,1229,1230,False,"[453, 1198, 1230]"


In [16]:
instance_df3 = instance_df2[['arc_u', 'arc_v', 'arc_cost']]

In [17]:
network = Network(instance_df3)

In [18]:
network.update_network_df()

In [21]:
pd._testing.array_equivalent(network.network_df, instance_df2[['arc_u', 'arc_v', 'arc_cost', 'arc_index', 'arc_successor_index_list']])

False